In [1]:
# import the necessary libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [2]:
my_page = requests.get(
    "https://discovery.smartcolumbusos.com/?apiAccessible=true")
print(my_page.status_code) #200
#print(my_page.text) #prints whole output in byte format
#print(my_page.content) #prints whole output in unicode format

200


In [4]:
soup = BeautifulSoup(my_page.content, 'html.parser')

In [8]:
print (soup.prettify()[:])

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8">
   <meta content="width=device-width, initial-scale=1.0" name="viewport">
    <meta content="ie=edge" http-equiv="X-UA-Compatible">
     <title>
      Datasets
     </title>
     <link href="/favicon.ico" rel="shortcut icon">
      <link href="/2.a2f92c14aae5885c4b7f.css" rel="stylesheet">
       <link href="/0.a2f92c14aae5885c4b7f.css" rel="stylesheet"/>
      </link>
     </link>
    </meta>
   </meta>
  </meta>
 </head>
 <body>
  <section id="root">
  </section>
  <script src="/config.js" type="text/javascript">
  </script>
  <script src="/runtime.7680e7c1a7a03ac628f9.js" type="text/javascript">
  </script>
  <script src="/vendors.87491822db874fe28883.js" type="text/javascript">
  </script>
  <script src="/main.1da5c23e77dccdde435d.js" type="text/javascript">
  </script>
 </body>
</html>



In [ ]:
'https://discovery.smartcolumbusos.com/?apiAccessible=true'

In [120]:
df = pd.read_excel("Smart Columbus API data sources.xlsx")

In [88]:
def webscrape(site):

    from selenium import webdriver
    import time
    from selenium.webdriver.chrome.options import Options
    import re


    options = Options()
    options.headless = True

    EXE_PATH = r'/Applications/chromedriver'
    driver = webdriver.Chrome(executable_path=EXE_PATH, options=options)
    driver.set_window_size(1440, 900)
    driver.get(site)
    time.sleep(5)

    last_modified= []
    file_type = []
    description = []
    url_final = []
    title_final =[]

    for a in driver.find_elements_by_xpath(".//div[@class='result-count']"):
        num = str(a.text)
    num = re.sub("[^0-9]", "", num)
    num2 = round(int(num)/10,0)+1
    j=0
    
    for x in range(1,int(num2)):

        site = website+str(x)
        url_temp = []
        title_temp = []
        
        driver.get(site)
        time.sleep(5)
        z=0
        for a in driver.find_elements_by_class_name('title'):
            z+=1
            if z>1:
                url_final.append(str(a.get_attribute('href')))
                title_final.append(str(a.text))

        for a in driver.find_elements_by_xpath(".//div[@class='last-modified']"):
            last_modified.append(str(a.text))

        for a in driver.find_elements_by_xpath(".//span[@class='file-type']"):
            file_type.append(str(a.text))

        for a in driver.find_elements_by_xpath(".//div[@class='description']"):
            description.append(str(a.text))
        
        time.sleep(5)
        j+=1
        if j%20==0:
            print(str(j)," pages scraped, ", str(len(url_final)), " datasets collected")

    driver.close()
    return     last_modified, file_type , description , url_final ,title_final



In [97]:
from datetime import date

for x in ['https://discovery.smartcolumbusos.com/?q=&sort=name_asc&apiAccessible=true&page=1',
         'https://discovery.smartcolumbusos.com/?q=&sort=name_asc&apiAccessible=false&page=1']:
    last_modified, file_type , description , url_final ,title_final = webscrape(x)

    df_temp = pd.DataFrame({
        'Title':title_final,
        'File Type':file_type,
        'Last Modified Date':last_modified,
        'Description':description,
        'URL':url_final
    }) 
    
    agency = []
    new_dates = []

    for x in df_temp['URL']:
        y = x[46:]
        z=y[:y.find("/")]
        agency.append(z)

    for x in df_temp['Last Modified Date']:
        y = x[:12]
        if y == "Date not pro":
            y=""
        if y == "Updates to r":
            y=""
        new_dates.append(y)
    
    df_temp['Agency']=agency    
    df_temp['Last Modified Date New']=new_dates
    df_temp['date pulled']=date.today()
    
    df=df.append(df_temp)
    print("done")

20  pages scraped,  200  datasets collected
40  pages scraped,  400  datasets collected
60  pages scraped,  600  datasets collected


NameError: name 'j' is not defined

In [109]:
len(df)

3800

In [117]:
df.head()

,Agency,Description,File Type,Last Modified Date,Last Modified Date New,Title,URL,date pulled
0,ogrip,Cuyahoga County Planning Commission's Greenpri...,CSV,Updates to remote datasets are not tracked,Updates to r,100 Year Flood Plain -- CSV,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
1,ogrip,Cuyahoga County Planning Commission's Greenpri...,GEOJSON,Updates to remote datasets are not tracked,Updates to r,100 Year Flood Plain -- GeoJSON,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
2,ogrip,Cuyahoga County Planning Commission's Greenpri...,KML,Updates to remote datasets are not tracked,Updates to r,100 Year Flood Plain -- KML,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
3,ogrip,Cuyahoga County Planning Commission's Greenpri...,OGC WMS,Updates to remote datasets are not tracked,Updates to r,100 Year Flood Plain -- OGC WMS,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
4,ogrip,Cuyahoga County Planning Commission's Greenpri...,SHAPEFILE,Date not provided,NaN,100 Year Flood Plain -- Shapefile,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06


In [113]:
df.Agency.value_counts()

ogrip                                   3217
city_of_columbus                         200
morpc                                    111
central_ohio_transit_authority            54
odot                                      45
handson_central_ohio                      36
may_mobility                              20
united_states_census_bureau               17
ohio_department_of_health                 14
aashto                                    14
geotab                                    14
urban_regional_analysis                    8
us_dept_agriculture                        8
ips_group                                  6
columbus_public_health                     5
conduent                                   4
mid_ohio_food_bank                         3
center_for_disease_control                 3
noaa                                       2
cogo                                       2
franklin_cnty_municipal_court              2
kirwan_institute                           2
scos      

In [121]:
df.sort_values("Last Modified Date New", ascending=False).head()

,Agency,Description,File Type,Last Modified Date,Last Modified Date New,Title,URL,date pulled
2428,may_mobility,A rolling snapshot of data collected by the co...,JSON,"Sep 30, 2019 7:47 AM (Last Ingested)","Sep 30, 2019",Connected Electric Autonomous Vehicle Locations,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
2429,may_mobility,A rolling snapshot of data collected by the co...,JSON,"Sep 30, 2019 7:47 AM (Last Ingested)","Sep 30, 2019",Connected Electric Autonomous Vehicle Locations,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
2430,central_ohio_transit_authority,This dataset identifies unlinked passenger tri...,CSV,"Sep 3, 2019 (Last updated by provider)","Sep 3, 2019",COTA Unlinked Passenger Trips Data for Saturda...,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
2434,central_ohio_transit_authority,This dataset identifies unlinked passenger tri...,CSV,"Sep 3, 2019 (Last updated by provider)","Sep 3, 2019",COTA Unlinked Passenger Trips Data for Sunday ...,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06
2435,central_ohio_transit_authority,This dataset identifies unlinked passenger tri...,CSV,"Sep 3, 2019 (Last updated by provider)","Sep 3, 2019",COTA Unlinked Passenger Trips Data for Weekday...,https://discovery.smartcolumbusos.com/dataset/...,2019-11-06


In [107]:
df.to_excel("Smart Columbus API data sources.xlsx", index=False)